## ML model testing

* logistic / multinomial logistic regression



In [150]:
import pandas as pd
pd.set_option('display.max_columns',50)
#pd.set_option('display.width',1000)
pd.set_option('display.max_rows', 50)
import numpy as np
from numpy import argmax
import datetime as dt
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
sns.set_theme(style='whitegrid')

from sklearn.preprocessing import StandardScaler, MinMaxScaler, OrdinalEncoder
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.linear_model import LogisticRegression, LinearRegression
import xgboost as xgb
from sklearn.metrics import r2_score, mean_squared_error
from scipy.stats import norm
from sklearn.utils import resample
from sklearn import linear_model

from collections import Counter

In [151]:
df = pd.read_csv('/Users/wjdol/Desktop/LighthouseLabs/Flight_delays/data/train_flights_complete_raw200.csv', parse_dates=['fl_date'])

In [9]:
# df['fl_day'] = df['fl_date'].dt.weekday

In [152]:
df.head()

,Unnamed: 0,mkt_unique_carrier,origin,origin_city,dest,dest_city,crs_elapsed_time,crs_dep_hour,dep_tod,crs_arr_hour,arr_tod,distance,distance_group,fl_date,fl_day,week_num,departures_performed,payload,passengers,freight,sdomt_gallons,tdomt_gallons,sdomt_cost,tdomt_cost,AWND,PRCP,SNOW,SNWD,TAVG,arr_delay,arr_delay_pos,delay_binary
0,0,UA,ORD,Chicago,MHT,Manchester,140.0,11,morning,14,afternoon,843.0,2.0,2018-01-21,Sunday,3,24.0,306500.0,668.0,0.0,6214610.8,6214610.8,12885113.2,12885113.2,17.0,0.0,0.0,0.0,46.0,18.0,18.0,1.0
1,1,WN,AUS,Austin,DAL,Dallas,55.0,22,evening,23,evening,189.0,1.0,2018-01-14,Sunday,2,66.5,2289000.0,6798.0,4205.5,156295770.8,156532592.0,293397830.2,293803944.4,13.0,0.0,0.0,0.0,33.0,-11.0,0.0,0.0
2,2,AA,DFW,Dallas/Fort Worth,IND,Indianapolis,120.0,15,afternoon,18,evening,761.0,2.0,2018-01-04,Thursday,1,119.5,4572500.0,13895.5,4076.5,162609079.0,163140817.6,264949334.8,265791415.6,15.0,0.0,0.0,0.0,38.0,7.0,7.0,1.0
3,4,AS,SEA,Seattle,PDX,Portland,45.0,15,afternoon,16,afternoon,129.0,1.0,2018-01-28,Sunday,4,466.0,8285900.0,23768.0,13757.0,0.0,0.0,0.0,0.0,38.0,15.0,0.0,0.0,83.0,-4.0,0.0,0.0
4,5,DL,LGA,New York,STL,St. Louis,190.0,19,evening,21,evening,888.0,2.0,2018-01-26,Friday,4,71.0,1526642.0,3093.0,0.0,0.0,0.0,0.0,0.0,30.0,0.0,0.0,0.0,0.0,-25.0,0.0,0.0


In [153]:
# drop columns not used

df = df.drop(['Unnamed: 0', 'origin', 'dest', 'fl_date', 'delay_binary', 'distance', 'sdomt_gallons', 'sdomt_cost', 'arr_delay', 'crs_dep_hour', 'crs_arr_hour', 'SNWD'], axis=1)

In [154]:
df['week_num'] = df.week_num.astype('object')

In [155]:
df.loc[df['tdomt_gallons']==0, 'tdomt_gallons'] = np.mean(df['tdomt_gallons'])

In [156]:
df.loc[df['tdomt_cost']==0,'tdomt_cost'] = np.mean(df['tdomt_cost'])

In [157]:

df = df.drop(df[df['passengers']==0].index)

In [158]:
dflreg = df.copy()

In [159]:
# bins for numerical data

dflreg.loc[dflreg['SNOW']>0,'SNOW']=1
# dflreg['SNWD'] = pd.cut(dflreg['SNWD'], bins=[0, 0.1, 60, 150, np.inf], labels=['0','1','2','3'], include_lowest=True)

dflreg.loc[dflreg['PRCP']>0,'PRCP']=1
# dflreg['PRCP'] = pd.cut(dflreg['PRCP'], bins=[0, 0.1, 75, 250, np.inf], labels=['0','1','2','3'], include_lowest=True)

In [149]:
# dflreg['SNOW'] = dflreg.SNOW.astype('int')
# dflreg['PRCP'] = dflreg.PRCP.astype('int')

In [160]:
# make bins for grouping

airlines_list = dict(pd.qcut(dflreg['mkt_unique_carrier'].value_counts(), q=[0, 0.45, 0.9, 1.0], labels=['1','2','3']))

# origin_list = dict(pd.qcut(dflreg['origin'].value_counts(), q=[0, 0.25, 0.5, 1.0], labels=['1','2','3']))

# dest_list = dict(pd.qcut(dflreg['dest'].value_counts(), q=[0, 0.25, 0.5, 1.0], labels=['1','2','3']))


In [161]:
dflreg['mkt_unique_carrier'].replace(airlines_list, inplace=True)

# dflreg['origin'].replace(origin_list, inplace=True)
# dflreg['dest'].replace(dest_list, inplace=True)

In [719]:
# create list of value counts by columns

# identifier_count = Counter(dict(dflreg['identifier'].value_counts()))
# carrier_count = Counter(dict(dflreg['mkt_unique_carrier'].value_counts()))
# #originc_count = Counter(dict(dflreg['origin_city'].value_counts()))
# origin_count = Counter(dict(dflreg['origin'].value_counts()))
# #destc_count = Counter(dict(dflreg['dest_city'].value_counts()))
# dest_count = Counter(dict(dflreg['dest'].value_counts()))
# wkday_count = Counter(dict(dflreg['fl_day'].value_counts()))
# #type_count = Counter(dict(dflreg['aircraft_type'].value_counts()))
# payl_count = Counter(dict(dflreg['payload'].value_counts()))

In [133]:
dflreg.head()

,mkt_unique_carrier,origin_city,dest_city,crs_elapsed_time,dep_tod,arr_tod,distance_group,fl_day,week_num,departures_performed,payload,passengers,freight,tdomt_gallons,tdomt_cost,AWND,PRCP,SNOW,TAVG,arr_delay_pos
0,2,Chicago,Manchester,140.0,morning,afternoon,2.0,Sunday,3,24.0,306500.0,668.0,0.0,6.214611e+06,1.288511e+07,17.0,0,0,46.0,18.0
1,2,Austin,Dallas,55.0,evening,evening,1.0,Sunday,2,66.5,2289000.0,6798.0,4205.5,1.565326e+08,2.938039e+08,13.0,0,0,33.0,0.0
2,3,Dallas/Fort Worth,Indianapolis,120.0,afternoon,evening,2.0,Thursday,1,119.5,4572500.0,13895.5,4076.5,1.631408e+08,2.657914e+08,15.0,0,0,38.0,7.0
3,2,Seattle,Portland,45.0,afternoon,afternoon,1.0,Sunday,4,466.0,8285900.0,23768.0,13757.0,9.079810e+07,1.631159e+08,38.0,1,0,83.0,0.0
4,2,New York,St. Louis,190.0,evening,evening,2.0,Friday,4,71.0,1526642.0,3093.0,0.0,9.079810e+07,1.631159e+08,30.0,0,0,0.0,0.0


In [104]:
dflreg.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 149470 entries, 0 to 149473
Data columns (total 20 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   mkt_unique_carrier    149470 non-null  object 
 1   origin_city           149470 non-null  object 
 2   dest_city             149470 non-null  object 
 3   crs_elapsed_time      149470 non-null  float64
 4   dep_tod               149470 non-null  object 
 5   arr_tod               149470 non-null  object 
 6   distance_group        149470 non-null  float64
 7   fl_day                149470 non-null  object 
 8   week_num              149470 non-null  object 
 9   departures_performed  149470 non-null  float64
 10  payload               149470 non-null  float64
 11  passengers            149470 non-null  float64
 12  freight               149470 non-null  float64
 13  tdomt_gallons         149470 non-null  float64
 14  tdomt_cost            149470 non-null  float64
 15  

In [162]:
# add new features based on fuel consumption, fuel cost, and passengers

dflreg['costPerGallon'] = dflreg['tdomt_cost']/dflreg['tdomt_gallons']
dflreg['costPerPerson'] = dflreg['tdomt_cost']/dflreg['passengers']
dflreg['gallonPerPerson'] = dflreg['tdomt_gallons']/dflreg['passengers']


In [645]:
# determine quartile splits

# top_carriers = top_categories(carrier_count, 0.90)
# top_dest = top_categories(dest_count, 0.5)
# top_origin = top_categories(origin_count, 0.5)
# top_identifier = top_categories(identifier_count, 0.05)

In [135]:
dflreg.head()

,mkt_unique_carrier,origin_city,dest_city,crs_elapsed_time,dep_tod,arr_tod,distance_group,fl_day,week_num,departures_performed,payload,passengers,freight,tdomt_gallons,tdomt_cost,AWND,PRCP,SNOW,TAVG,arr_delay_pos,costPerGallon,costPerPerson,gallonPerPerson
0,2,Chicago,Manchester,140.0,morning,afternoon,2.0,Sunday,3,24.0,306500.0,668.0,0.0,6.214611e+06,1.288511e+07,17.0,0,0,46.0,18.0,2.073358,19289.091617,9303.309581
1,2,Austin,Dallas,55.0,evening,evening,1.0,Sunday,2,66.5,2289000.0,6798.0,4205.5,1.565326e+08,2.938039e+08,13.0,0,0,33.0,0.0,1.876951,43219.173934,23026.271256
2,3,Dallas/Fort Worth,Indianapolis,120.0,afternoon,evening,2.0,Thursday,1,119.5,4572500.0,13895.5,4076.5,1.631408e+08,2.657914e+08,15.0,0,0,38.0,7.0,1.629215,19127.877054,11740.550365
3,2,Seattle,Portland,45.0,afternoon,afternoon,1.0,Sunday,4,466.0,8285900.0,23768.0,13757.0,9.079810e+07,1.631159e+08,38.0,1,0,83.0,0.0,1.796468,6862.836669,3820.182668
4,2,New York,St. Louis,190.0,evening,evening,2.0,Friday,4,71.0,1526642.0,3093.0,0.0,9.079810e+07,1.631159e+08,30.0,0,0,0.0,0.0,1.796468,52737.116698,29355.997944


In [136]:
dflreg.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 149470 entries, 0 to 149473
Data columns (total 23 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   mkt_unique_carrier    149470 non-null  object 
 1   origin_city           149470 non-null  object 
 2   dest_city             149470 non-null  object 
 3   crs_elapsed_time      149470 non-null  float64
 4   dep_tod               149470 non-null  object 
 5   arr_tod               149470 non-null  object 
 6   distance_group        149470 non-null  float64
 7   fl_day                149470 non-null  object 
 8   week_num              149470 non-null  object 
 9   departures_performed  149470 non-null  float64
 10  payload               149470 non-null  float64
 11  passengers            149470 non-null  float64
 12  freight               149470 non-null  float64
 13  tdomt_gallons         149470 non-null  float64
 14  tdomt_cost            149470 non-null  float64
 15  

In [163]:
dflreg = dflreg.drop(['payload', 'passengers', 'freight', 'tdomt_gallons', 'tdomt_cost'], axis=1)

In [164]:
dflreg = pd.get_dummies(dflreg, columns=['dep_tod','arr_tod', 'origin_city', 'dest_city', 'week_num', 'fl_day'], drop_first=True)

C:\Users\wjdol\Desktop\LighthouseLabs\env\lib\site-packages\pandas\core\algorithms.py:798: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  uniques = Index(uniques)


In [165]:
# split features and target

y = dflreg['arr_delay_pos']
X = dflreg.drop('arr_delay_pos', axis=1)

In [166]:
# train / test split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [167]:
num_X_train = X_train.select_dtypes(include=['float64'])
num_X_test = X_test.select_dtypes(include=['float64'])

In [168]:
num_X_train

,crs_elapsed_time,distance_group,departures_performed,AWND,PRCP,SNOW,TAVG,costPerGallon,costPerPerson,gallonPerPerson
14474,85.0,1.0,74.5,17.0,0.0,0.0,22.0,1.876951,35950.314396,19153.575038
146017,99.0,1.0,43.0,102.0,0.0,0.0,14.0,1.796468,149510.450914,83224.657783
86745,332.0,5.0,38.0,25.0,0.0,0.0,231.0,1.629215,45118.216873,27693.229944
43336,294.0,4.0,3.0,75.0,0.0,0.0,-117.0,1.667303,957224.097030,574115.228713
35744,140.0,2.0,12.0,36.0,0.0,0.0,156.0,1.629215,168542.432213,103450.106278
...,...,...,...,...,...,...,...,...,...,...
82578,107.0,1.0,60.0,42.0,0.0,0.0,43.0,1.629215,43658.248292,26797.111958
120277,160.0,2.0,13.0,29.0,0.0,1.0,-274.0,1.929986,142622.021656,73897.973103
99803,126.0,2.0,17.0,26.0,0.0,0.0,-72.0,1.929986,138712.689974,71872.397499
121739,152.0,2.0,31.0,20.0,0.0,0.0,92.0,1.629215,78036.234762,47898.067410


In [1198]:
# dflreg.isnull().sum()

In [169]:
cat_X_train = X_train.select_dtypes(exclude=['float64'])
cat_X_test = X_test.select_dtypes(exclude=['float64'])

In [170]:
scaler = StandardScaler()

scaler.fit(num_X_train)

scaled_X_train = scaler.transform(num_X_train)
scaled_X_test = scaler.transform(num_X_test)


In [171]:
cat_X_train_f = cat_X_train.to_numpy()
cat_X_test_f = cat_X_test.to_numpy()

In [172]:
X_train_f = np.hstack((cat_X_train_f, scaled_X_train))
X_test_f = np.hstack((cat_X_test_f, scaled_X_test))

In [173]:
# create xgb initial model

xgb_0 = xgb.XGBRegressor(n_estimators=100, max_depth=10, alpha=0.1, colsample_bytree=0.3, objective='reg:squarederror')

xgb_0.fit(X_train_f, y_train)

XGBRegressor(alpha=0.1, base_score=0.5, booster='gbtree', callbacks=None,
             colsample_bylevel=1, colsample_bynode=1, colsample_bytree=0.3,
             early_stopping_rounds=None, enable_categorical=False,
             eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
             importance_type=None, interaction_constraints='',
             learning_rate=0.300000012, max_bin=256, max_cat_to_onehot=4,
             max_delta_step=0, max_depth=10, max_leaves=0, min_child_weight=1,
             missing=nan, monotone_constraints='()', n_estimators=100, n_jobs=0,
             num_parallel_tree=1, predictor='auto', random_state=0,
             reg_alpha=0.100000001, ...)

In [174]:
y_pred = xgb_0.predict(X_test_f)
r2_score(y_test, y_pred)

0.008099154183862423

In [50]:
params = {'max_depth': [10, 15, 20],
           'learning_rate': [0.01, 0.1, 0.2],
           'subsample': np.arange(0.7, 1.0, 0.1),
           'colsample_bytree': np.arange(0.4, 0.8, 0.1),
           'colsample_bylevel': np.arange(0.4, 1.0, 0.1),
           'n_estimators': [500, 1000, 1500]}

xgb_1 = xgb.XGBRegressor()

reg_1 = RandomizedSearchCV(estimator=xgb_1, param_distributions=params, scoring='r2', n_iter=1, cv=2)

In [51]:
reg_1.fit(X_train_f, y_train)

RandomizedSearchCV(cv=2,
                   estimator=XGBRegressor(base_score=None, booster=None,
                                          callbacks=None,
                                          colsample_bylevel=None,
                                          colsample_bynode=None,
                                          colsample_bytree=None,
                                          early_stopping_rounds=None,
                                          enable_categorical=False,
                                          eval_metric=None, gamma=None,
                                          gpu_id=None, grow_policy=None,
                                          importance_type=None,
                                          interaction_constraints=None,
                                          learning_rate=None, max_bin=None,
                                          m...
                                          num_parallel_tree=None,
                                        

In [52]:
reg_1.best_params_


{'subsample': 0.9999999999999999,
 'n_estimators': 1500,
 'max_depth': 20,
 'learning_rate': 0.1,
 'colsample_bytree': 0.5,
 'colsample_bylevel': 0.7999999999999999}

In [53]:
reg_1.best_score_

-0.0115609402257697

In [47]:
reg_1.cv_results_

{'mean_fit_time': array([139.16542244]),
 'std_fit_time': array([5.51408832]),
 'mean_score_time': array([0.97981963]),
 'std_score_time': array([0.04293757]),
 'param_subsample': masked_array(data=[0.7999999999999999],
              mask=[False],
        fill_value='?',
             dtype=object),
 'param_n_estimators': masked_array(data=[500],
              mask=[False],
        fill_value='?',
             dtype=object),
 'param_max_depth': masked_array(data=[15],
              mask=[False],
        fill_value='?',
             dtype=object),
 'param_learning_rate': masked_array(data=[0.2],
              mask=[False],
        fill_value='?',
             dtype=object),
 'param_colsample_bytree': masked_array(data=[0.6],
              mask=[False],
        fill_value='?',
             dtype=object),
 'param_colsample_bylevel': masked_array(data=[0.7999999999999999],
              mask=[False],
        fill_value='?',
             dtype=object),
 'params': [{'subsample': 0.79999999999